In [1]:
import json
import re
from pathlib import Path
from collections import Counter

import pandas as pd

AGENT_DIR = Path("/workspace/model-organisms/diffing_results/gemma3_1B/cake_bake/activation_difference_lens/agent")

In [2]:
# Parse directory name: {AgentType}_{LLM}_{miN}_run{R}
DIR_PATTERN = re.compile(r"^(?P<agent_type>.+?)_(?P<llm>openai_.+?)_mi(?P<mi_budget>\d+)_run(?P<run>\d+)$")
# Parse CALL(tool_name: ...) from assistant messages
CALL_PATTERN = re.compile(r"^CALL\((?P<tool>\w+):")

ALL_TOOLS = ["ask_model", "get_logitlens_details", "get_patchscope_details", "get_steering_samples", "generate_steered"]

rows = []
for run_dir in sorted(AGENT_DIR.iterdir()):
    if not run_dir.is_dir():
        continue
    m = DIR_PATTERN.match(run_dir.name)
    if not m:
        print(f"Skipping unrecognized dir: {run_dir.name}")
        continue

    agent_type = m.group("agent_type")
    llm = m.group("llm")
    mi_budget = int(m.group("mi_budget"))
    run_idx = int(m.group("run"))

    # Load messages
    messages = json.loads((run_dir / "messages.json").read_text())

    # Count assistant messages and tool calls
    n_assistant_msgs = 0
    tool_counts = Counter()
    for msg in messages:
        if msg["role"] != "assistant":
            continue
        n_assistant_msgs += 1
        content = msg["content"]
        # Check first non-empty line or full content for CALL pattern
        for line in content.strip().splitlines():
            line = line.strip()
            call_match = CALL_PATTERN.match(line)
            if call_match:
                tool_counts[call_match.group("tool")] += 1

    # Load stats
    stats = json.loads((run_dir / "stats.json").read_text())
    mi_used = stats.get("model_interactions_used", 0)

    # Load judge scores
    grade_files = sorted(run_dir.glob("hypothesis_grade_*.json"))
    scores = []
    for gf in grade_files:
        grade = json.loads(gf.read_text())
        scores.append(grade["score"])
    scores_str = ",".join(str(s) for s in scores)

    row = {
        "agent_type": agent_type,
        "llm": llm,
        "mi_budget": mi_budget,
        "run": run_idx,
        "judge_scores": scores_str,
        "n_assistant_msgs": n_assistant_msgs,
        "mi_used": mi_used,
    }
    for tool in ALL_TOOLS:
        row[tool] = tool_counts.get(tool, 0)
    rows.append(row)

df = pd.DataFrame(rows)
df

,agent_type,llm,mi_budget,run,judge_scores,n_assistant_msgs,mi_used,ask_model,get_logitlens_details,get_patchscope_details,get_steering_samples,generate_steered
0,ADL,openai_gpt-5,0,0,"1,1,2",2,0,1,0,0,0,0
1,ADL,openai_gpt-5,0,1,"2,1,2",2,0,1,0,0,0,0
2,ADL,openai_gpt-5,0,2,"1,2,2",2,0,1,0,0,0,0
3,ADL,openai_gpt-5,0,3,"2,1,1",2,0,1,0,0,0,0
4,ADL,openai_gpt-5,0,4,"2,2,2",2,0,1,0,0,0,0
5,ADL,openai_gpt-5,5,0,"2,2,2",7,5,5,0,0,0,0
6,ADL,openai_gpt-5,5,1,"2,2,2",6,5,5,0,0,0,0
7,ADL,openai_gpt-5,5,2,"4,4,2",4,5,2,0,0,0,0
8,ADL,openai_gpt-5,5,3,"1,1,1",6,5,5,0,0,0,0
9,ADL,openai_gpt-5,5,4,"1,2,2",5,5,4,0,0,0,0
